In [207]:
from draco.learn import linear
from draco.learn import data_util
from draco.run import run
from draco.spec import Task, Query
from draco.learn.helper import count_violations
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [208]:
test_size = 0.3

In [209]:
data, _ = data_util.load_data(test_size=test_size)
data = data[data.source == 'younghoon']

raw_data = data_util.load_neg_pos_specs()

In [210]:
X = data.positive - data.negative
y = np.ones(len(X))

In [211]:
clf = linear.train_model(X, test_size=test_size)
pred = clf.predict(X)

Train score:  0.9273049645390071
Dev score:  0.9259259259259259


In [212]:
weights = clf.coef_[0]
names = X.columns

In [224]:
const_dict = dict(zip(map(lambda x: f'{x}_weight', names), map(int, weights*1000)))
weight_dict = dict(zip(names, weights))

const_dict, weight_dict;

In [214]:
only_correct = X[pred == 1]

len(only_correct), len(data)

(748, 807)

In [223]:
def aggregate(violations):
    ''' Get the cost from a dictionary of violations. '''
    ret = 0.0
    for n, v in violations.items():
        ret += weight_dict[n] * v
    return ret

for i in range(len(only_correct))[:10]:
    name = only_correct.iloc[i].name
    
    print(name)
    
    pair = raw_data[name]

    task_n = Task(pair.data, Query.from_vegalite(pair.negative),task=pair.task)
    task_p = Task(pair.data, Query.from_vegalite(pair.positive),task=pair.task)
    
    assert clf.predict([entry]) == 1
    
    data_entry = data.loc[name]
    
    # print("Prediction negative", clf.coef_.dot(np.array([data_entry.negative]).T))
    print("Prediction negative:", clf.decision_function([data_entry.negative])[0])
    # print("Prediction positive", clf.coef_.dot(np.array([data_entry.positive]).T))
    print("Prediction positive:", clf.decision_function([data_entry.positive])[0])
    
#     cn = run(task_n, constants=const_dict).cost
#     cp = run(task_p, constants=const_dict).cost
#     print(cn, cp, cn > cp)

    # the violation vectors have to be the same
    assert count_violations(task_n) == data_entry.negative[data_entry.negative>0].to_dict()

    cn = aggregate(count_violations(task_n))
    cp = aggregate(count_violations(task_p))
    
    print(cn, cp, cn > cp)
    
    print()

younghoon-664
Prediction negative: -0.4690164336214597
Prediction positive: 0.9844357445817346
-0.4690164336214597 0.9844357445817346 False

younghoon-955
Prediction negative: -1.1593905358333618
Prediction positive: 0.879401919499872
-1.1593905358333618 0.879401919499872 False

younghoon-1016
Prediction negative: -1.1593905358333618
Prediction positive: 1.1289212394085673
-1.1593905358333618 1.128921239408567 False

younghoon-843
Prediction negative: -1.1593905358333618
Prediction positive: 1.1289212394085673
-1.1593905358333618 1.128921239408567 False

younghoon-327
Prediction negative: 0.25209967285299684
Prediction positive: 0.7449093623739602
0.25209967285299695 0.7449093623739601 False

younghoon-1043
Prediction negative: -0.4068890889129172
Prediction positive: 0.9120360481370211
-0.40688908891291714 0.9120360481370211 False

younghoon-991
Prediction negative: -0.6580620622887378
Prediction positive: 0.9120360481370211
-0.6580620622887378 0.9120360481370211 False

younghoon-689
